## Notebook in how to plot the evolution of an impact crater in iSALE

A routine in Python has been developed in order to quickly plot results obtained from iSALE. A lot of work is required to improve those functions and make them more user-friendly.

**Summary of functions:** <br>
make_a_video

**Possible upgrades:** <br>
- blabla
- blabla2

**History:**<br>
Original version (1.0):<br>
2017/08/XX<br>

Improved version (2.0):<br>
2017/22/11<br>    

In [1]:
# importing both Python's and local modules
import numpy as np
import os
import sys
import copy
import glob
import subprocess

# matplotlib related import
import matplotlib.pyplot as plt
from pylab import arange
from matplotlib import rc
import matplotlib.gridspec as gridspec

# Switch on the use of Latex text
rc('text', usetex=True)
rc('axes', linewidth=2)
rc('font', weight='bold')

# local modules
pathm = ['/work/nilscp/Python/prog/clean', '/work/nilscp/iSALE/Dellen/lib']

for pat in pathm:
    sys.path.append(pat)

import pySALEPlot as psp # from /work/nilscp/iSALE/Dellen/lib
import crater as cr # from /work/nilscp/Python/prog/clean

       - pySALEPlot -
       by Tom Davison


In [10]:
# functions

'''
***********************************************************************
'''

def make_video(path, delay_number, loop_number, videolabel):
    '''
    description:

    path = '/mypath/' # note the slash at the end
    delay_number: in hundreds of a second
    loop: = 0 infinite looping, 1 = once, 2 = twice ....
    videoname = 'myvideo.gif'  

    example:
    path = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/CDILWPO_L100/evolution/plots/Distension/all/'
    delay_number = 10
    loop_number = 1
    videoname = 'distension'
    make_video(path, delay_number, loop_number, videoname)
    '''
    os.chdir(path)

    videoname = videolabel + '_delay' + \
        str(int(delay_number)) + '_loop' + str(loop_number) + '.gif'

    command = ("convert -delay " + str(delay_number) + " -loop " + str(loop_number) + " " + str(path) +
               "*.png " + videoname)

    subprocess.Popen(command.split(), cwd=path)

    print ('Steven Spielberg')


'''
***********************************************************************
'''


def find_nearest(array, value):
    '''
    description:
    find the nearest value in an array

    output:
    value and index where the nearest value is found
    '''

    idx = np.nanargmin((np.abs(array-value)))
    return array[idx], idx


'''
***********************************************************************
'''


def morphology(path1, paths, norm, normpath, timei, showtransient, showhline, lbl):
    '''    
    path1 = '/work/nilscp/iSALE/isaleruns/testruns/ejecta/L20km_part1_HYDRO_CPPR1000/results/L20km_HYDRO_part1g/'
    paths = '/work/nilscp/iSALE/isaleruns/data/ejecta/L20km_HYDRO_part1g/'
    normpath = '/work/nilscp/iSALE/isaleruns/data/ejecta/L20km_HYDRO_part1g/'
    norm = 0.5 # 0.5 by the radius of the projectile?
    showtransient = False
    showhline = False
    timei = 'pen' #'transient' or 'norm' or 'pen' (for penetration time)

    lbl = r"$\mathit{U} = 12.7 km/s, \mathit{L} = 20 km$"

    morphology(path1, paths, norm, normpath, timei, showtransient, showhline, lbl)



    #loop
    L = [100,200,300,400,500,600,700,800,900,1000]
    name = 'C00P10F06LONG_U04_L'

    for l in L:
        tmp_name = name + str(l)
         path1 = '/uio/kant/geo-ceed-u1/nilscp/Desktop/stallo_work/velocity/results/' + tmp_name
         paths = '/run/media/nilscp/Zell/velocity/data/'  + tmp_name
         lbl = r'$\mathit{U} = 400 m/s, \mathit{L} = ' + str(l) + ' m$"
         evoplot(path1,paths, path1tr, pathstr, norm, extentx, extenty, lbl)


    ############################3

    What if it is nan?

    What if we want to plot the transient crater in addition

    Maybe I could also have a function without having the need to have extentx and extenty
    taking 20% of the final drim and so much of the maximum depth.... or a fraction of the final drim
    we known that d ~ 0.2D

    norm = 0.5 can only be used for showtransient == False
    '''

    plt.ioff()  # figures does not pop up

    os.chdir(path1)
    mod1 = psp.opendatfile('jdata.dat')

    modelname = path1.split('/')[-2]

    if not os.path.exists(paths + 'evolution/plots/morphology'):
        os.makedirs(paths + 'evolution/plots/morphology')

    if (norm == 0):
        if not os.path.exists(paths + 'evolution/plots/morphology/absolute/'):
            os.makedirs(paths + 'evolution/plots/morphology/absolute')
    elif (norm == 0.5):
        if not os.path.exists(paths + 'evolution/plots/morphology/norm_proj/'):
            os.makedirs(paths + 'evolution/plots/morphology/norm_proj')
    elif (norm == 1):
        if not os.path.exists(paths + 'evolution/plots/morphology/norm_its/'):
            os.makedirs(paths + 'evolution/plots/morphology/norm_its')
    else:
        if not os.path.exists(paths + 'evolution/plots/morphology/norm/'):
            os.makedirs(paths + 'evolution/plots/morphology/norm')

    if norm == 1:
        os.chdir(paths + 'transient/')

        # load data for the transient crater 1
        t_trg, da_tr1, Da_tr1, V_tr1, h_tr1, Dr_tr1, Vr_tr1, __ = np.loadtxt(
            modelname + '_tr.txt', delimiter='\t', comments='#')

        os.chdir(paths + 'final/')
        t_f, __, __, __, __, drim_f, __ = np.loadtxt(
            modelname + '_final.txt', delimiter='\t', comments='#')
        rinner1 = drim_f/2.

    elif norm == 2:

        os.chdir(paths + 'transient/')

        # load data for the transient crater
        t_trg, __, __, __, __, __, __, __ = np.loadtxt(
            modelname + '_tr.txt', delimiter='\t', comments='#')

        modelnametr = normpath.split('/')[-2]
        os.chdir(normpath + 'transient/')

        # load data for the transient crater modtr
        t_tr1, da_tr1, Da_tr1, V_tr1, h_tr1, Dr_tr1, Vr_tr1, __ = np.loadtxt(
            modelnametr + '_tr.txt', delimiter='\t', comments='#')

        os.chdir(normpath + 'final/')
        t_fnorm, __, __, __, __, drim_fnorm, __ = np.loadtxt(
            modelnametr + '_final.txt', delimiter='\t', comments='#')
        rinner1 = drim_fnorm/2.

    else:
        os.chdir(paths + 'transient/')

        # load data for the transient crater 1
        t_trg, da_tr1, Da_tr1, V_tr1, h_tr1, Dr_tr1, Vr_tr1, __ = np.loadtxt(
            modelname + '_tr.txt', delimiter='\t', comments='#')

        os.chdir(paths + 'final/')
        t_f, __, __, __, __, drim_f, __ = np.loadtxt(
            modelname + '_final.txt', delimiter='\t', comments='#')

    # if we want to show the transient crater
    if showtransient:
        os.chdir(paths + 'transient/')
        dataXY = np.loadtxt(
            modelname + '_XYtransientprofile.txt', delimiter='\t', comments='#')
        X = dataXY[:, 0]
        Y = dataXY[:, 1]

    os.chdir(path1)
    for i in range(mod1.nsteps):
        step = mod1.readStep('Den', i)

        t1 = step.time
        # plotting of the data
        fig = plt.figure(figsize=(6, 6))
        ax1 = fig.add_subplot(111)

        if norm >= 1:
            ax1.pcolormesh(mod1.x/rinner1, mod1.y/rinner1, step.mat,
                           cmap='BrBG_r', vmin=-5., vmax=5., zorder=2)
            ax1.contour(mod1.xc/rinner1, mod1.yc/rinner1,
                        step.cmc[0], 1, colors='b', linewidths=4, zorder=4)
            if t1 >= t_trg:
                if showtransient:
                    # I need to show it only for a range
                    ax1.plot(X/rinner1, Y/rinner1, "ro")
                else:
                    None
            #ij1 = np.where((data_tr1[:,1]<=500.) & (data_tr1[:,0]<=Dr_tr1/2.))
            #ij2 = np.where((data_tr2[:,1]<=500.) & (data_tr2[:,0]<=Dr_tr2/2.))

            if showhline:
                ax1.hlines(0, 0, 1.2, 'r', linewidths=4, zorder=1)
            else:
                None

            ax1.set_xlim([0, 1.2])  # -2000,0 ## -1500,1500

            for ax in [ax1]:
                ax.minorticks_off()
                ax.tick_params('both', labelsize=16, length=10,
                               width=2., which='major')
                ax.set_ylim([-0.8, 0.8])

            for u in range(mod1.tracer_numu):
                tru = mod1.tru[u]
                # Plot the tracers in horizontal lines, every 20 lines
                for l in np.arange(0, len(tru.xlines), 20):
                    ax1.plot(step.xmark[tru.xlines[l]]/rinner1,
                             step.ymark[tru.xlines[l]]/rinner1,
                             c='k', marker='.', linestyle='None', markersize=1.0, zorder=3)

                # Plot the tracers in horizontal lines, every 20 lines
                for l in np.arange(0, len(tru.ylines), 20):
                    ax1.plot(step.xmark[tru.ylines[l]]/rinner1,
                             step.ymark[tru.ylines[l]]/rinner1,
                             c='k', marker='.', linestyle='None', markersize=1.0, zorder=3)

        elif norm == 0.5:
            ax1.pcolormesh(mod1.x/(mod1.cppr[0]*mod1.dx), mod1.y/(
                mod1.cppr[0]*mod1.dx), step.mat, cmap='BrBG_r', vmin=-5., vmax=5., zorder=2)
            ax1.contour(mod1.xc/(mod1.cppr[0]*mod1.dx), mod1.yc/(
                mod1.cppr[0]*mod1.dx), step.cmc[0], 1, colors='b', linewidths=4, zorder=4)
            if t1 >= t_trg:
                if showtransient:
                    # I need to show it only for a range
                    ax1.plot(X/(mod1.cppr[0]*mod1.dx),
                             Y/(mod1.cppr[0]*mod1.dx), "ro")
                else:
                    None

            if showhline:
                ax1.hlines(
                    0, 0, mod1.xhires[1]/(mod1.cppr[0]*mod1.dx), 'r', linewidths=4, zorder=1)
            else:
                None
            # -2000,0 ## -1500,1500
            ax1.set_xlim([0, mod1.xhires[1]/(mod1.cppr[0]*mod1.dx)])

            for ax in [ax1]:
                ax.set_ylim([mod1.yhires[0]/(mod1.cppr[0]*mod1.dy),
                             mod1.yhires[1]/(mod1.cppr[0]*mod1.dy)])
                ax.minorticks_off()
                ax.tick_params('both', labelsize=16, length=10,
                               width=2., which='major')

            for u in range(mod1.tracer_numu):
                tru = mod1.tru[u]
                # Plot the tracers in horizontal lines, every 20 lines
                for l in np.arange(0, len(tru.xlines), 20):
                    ax1.plot(step.xmark[tru.xlines[l]]/(mod1.cppr[0]*mod1.dx),
                             step.ymark[tru.xlines[l]]/(mod1.cppr[0]*mod1.dx),
                             c='k', marker='.', linestyle='None', markersize=1.0, zorder=3)

                # Plot the tracers in horizontal lines, every 20 lines
                for l in np.arange(0, len(tru.ylines), 20):
                    ax1.plot(step.xmark[tru.ylines[l]]/(mod1.cppr[0]*mod1.dx),
                             step.ymark[tru.ylines[l]]/(mod1.cppr[0]*mod1.dx),
                             c='k', marker='.', linestyle='None', markersize=1.0, zorder=3)

        else:
            ax1.pcolormesh(mod1.x, mod1.y, step.mat,
                           cmap='BrBG_r', vmin=-5., vmax=5., zorder=2)
            ax1.contour(mod1.xc, mod1.yc,
                        step.cmc[0], 1, colors='b', linewidths=4, zorder=4)
            if t1 >= t_trg:
                if showtransient:
                    ax1.plot(X, Y, "ro")  # I need to show it only for a range
                else:
                    None

            if showhline:
                ax1.hlines(0, 0, (drim_f/2.)*1.2, 'r', linewidths=4, zorder=1)
            else:
                None
            ax1.set_xlim([0, (drim_f/2.)*1.2])  # -2000,0 ## -1500,1500

            for ax in [ax1]:
                ax.set_ylim([-(drim_f/2.)*0.8, (drim_f/2.)*0.8])
                ax.minorticks_off()
                ax.tick_params('both', labelsize=16, length=10,
                               width=2., which='major')

            for u in range(mod1.tracer_numu):
                tru = mod1.tru[u]
                # Plot the tracers in horizontal lines, every 20 lines
                for l in np.arange(0, len(tru.xlines), 20):
                    ax1.plot(step.xmark[tru.xlines[l]],
                             step.ymark[tru.xlines[l]],
                             c='k', marker='.', linestyle='None', markersize=1.0, zorder=3)

                # Plot the tracers in horizontal lines, every 20 lines
                for l in np.arange(0, len(tru.ylines), 20):
                    ax1.plot(step.xmark[tru.ylines[l]],
                             step.ymark[tru.ylines[l]],
                             c='k', marker='.', linestyle='None', markersize=1.0, zorder=3)

        if norm == 0:
            ax1.set_xlabel("x (m)", fontsize=20)
            ax1.set_ylabel("y (m)", fontsize=20)
            fig.tight_layout()

        elif norm == 0.5:
            ax1.set_xlabel(r"$x / a$", fontsize=20)
            ax1.set_ylabel(r"$y / a$", fontsize=20)
            fig.tight_layout()

        else:
            ax1.set_xlabel(r"$x / R_{r}$", fontsize=20)
            ax1.set_ylabel(r"$y / R_{r}$", fontsize=20)
            fig.tight_layout()

        if timei == 'normal':
            ax1.set_title(r"$\mathit{t} = " + str(np.around(t1, decimals=2)
                                                  ) + " " + " s$", position=(0.5, 0.9), fontsize=18)
        elif timei == 'pen':
            ax1.set_title(r"$\mathit{t} = " + str(np.around(t1, decimals=2)) + " " + " s, $" + " " + "$t / t_s = $" + str(
                np.around(t1/((mod1.cppr[0]*mod1.dx*2.) / (mod1.objvel)), decimals=2)), position=(0.5, 0.9), fontsize=18)
        else:
            ax1.set_title(r"$\mathit{t} = " + str(np.around(t1, decimals=2)) + " " + " s, $" + " " +
                          "$\zeta$ = $" + str(np.around(t1/t_trg, decimals=2)), position=(0.5, 0.9), fontsize=18)

        st = fig.suptitle(lbl, fontsize=22)
        st.set_y(0.97)
        st.set_x(0.55)
        fig.subplots_adjust(top=0.9)

        if norm == 0:
            figtitle = modelname + '_' + str(int(i)).zfill(3) + ".png"
            fig.savefig(
                paths + 'evolution/plots/morphology/absolute/' + figtitle, dpi=300)
        elif norm == 0.5:
            figtitle = modelname + '_' + str(int(i)).zfill(3) + ".png"
            fig.savefig(
                paths + 'evolution/plots/morphology/norm_proj/' + figtitle, dpi=300)
        elif norm == 1:
            figtitle = 'itself_norm_' + modelname + \
                '_' + str(int(i)).zfill(3) + ".png"
            fig.savefig(
                paths + 'evolution/plots/morphology/norm_its/' + figtitle, dpi=300)
        else:
            figtitle = 'norm_' + modelname + \
                '_' + str(int(i)).zfill(3) + ".png"
            fig.savefig(
                paths + 'evolution/plots/morphology/norm/' + figtitle, dpi=300)
        plt.close()

    mod1.closeFile()


'''
***********************************************************************
'''


def zoom(zoom_id, norm, mod, rrim, manualx, manualy):
    '''
    example:
    zoom_id = 'hires' # 'close'; 'mid', 'hires'; 'all'
    norm = 2
    rrim = rinner1
    manualx = ((0,1500.)) # if norm not equal to 0 then it has to be in normalized distance!
    manualy = ((-1000., 500.)) # if norm not equal to 0 then it has to be in normalized distance!
    extentx, extenty = zoom(zoom_id, norm, mod1, rrim, manualx, manualy)
    
    added manual opportunity?
    
    if manual_entry
    '''
    
    # create empty arrays
    extentx = []
    extenty = []

    zid = np.array(['close', 'mid', 'hires', 'all', 'manual'])
    
    
    for z in zoom_id:

        idx = np.where(zid == z)[0][0]
    
        if norm == 0:
            if idx == 0:
                extentx.append((0.0, 1.2*rrim))
                extenty.append((-0.8*rrim, 0.8*rrim))
    
            elif idx == 1:
                extentx.append((0.0, 2.0*rrim))
                extenty.append((-2.0*rrim, 0.8*rrim))
    
            elif idx == 2:
                extentx.append((0.0, mod.xhires[1]))
                extenty.append((mod.yhires[0], mod.yhires[1]))
                
            elif idx == 4:
                extentx.append(manualx)
                extenty.append(manualy)
    
            else:
                extentx.append((0.0, np.max(mod.xc)))
                extenty.append((np.min(mod.yc), np.max(mod.yc)))
    
        else:
            if idx == 0:
                extentx.append((0.0, 1.2))
                extenty.append((-0.8, 0.8))
    
            elif idx == 1:
                extentx.append((0.0, 2.0))
                extenty.append((-2.0, 0.8))
    
            elif idx == 2:
                extentx.append((0.0, mod.xhires[1]/rrim))
                extenty.append((mod.yhires[0]/rrim, mod.yhires[1]/rrim))
                
            elif idx == 4:
                extentx.append(manualx)
                extenty.append(manualy)
    
            else:
                extentx.append((0.0, np.max(mod.xc)/rrim))
                extenty.append((np.min(mod.yc)/rrim, np.max(mod.yc)/rrim))
        
    ix_to_delete = []        
    for iy, y in enumerate(extenty):
        if np.isnan(extenty[iy][1]):
            ix_to_delete.append(iy)
                
    if len(ix_to_delete) > 0:
        zoom_id_up = np.delete(zoom_id, ix_to_delete)
            
        bb = np.array(range(len(zoom_id)))
        aa = np.array(ix_to_delete)
        cc = np.setdiff1d(bb,aa)
            
        extentxup = []
        extentyup = []
            
        for c in cc:
            extentxup.append(extentx[c])
            extentyup.append(extenty[c])
                
    else:
        extentxup = extentx
        extentyup = extenty
        zoom_id_up = zoom_id
            
    return extentxup, extentyup, zoom_id_up


'''
***********************************************************************
'''


def field_definition(model, zoom_id, paths, param = 'all'):
    '''
    example:
    param = 'Por'
    paths = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/CDILWPO_L100/'
    zoom_id = ['mid', 'hires']

    fld_param, fld_name, fld_cmap, fld_unit, fld_factor, npath = field_definition(model, zoom_id, paths, param = 'all')
    
    #Updated field_definition the 24th June 2019
    
    '''

    fld = np.array(['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld',
                    'Dam', 'Alp', 'TPS', 'YAc'])

    fld_param = np.array(['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld',
                          'Dam', 'Alp', 'TPS', 'YAc'])

    fld_name = np.array(['material 1', 'material 2', 'density', 'temperature', 'pressure', 'sie',
                         'YS', 'damage', 'distension', 'TPS',
                         'YAc'])

    fld_folder = np.array(['Cm1', 'Cm2', 'Density', 'Temperature', 'Pressure', 'Specific_internal_energy',
                           'Yield_strength', 'Damage', 'Distension', 'Total_plastic_strain',
                           'Acoustic_fluidisation_strength'])

    fld_cmap = np.array(['BrBG_r', 'BrBG_r', 'viridis', 'autumn', 'bwr', 'bwr',
                         'summer', 'winter', 'gray_r', 'Reds_r',
                         'summer'])

    fld_unit = np.array(['Conc. material','Conc. material','kg/m^{3}', 'Kelvin', 'GPa', 'MJ',
                         'GPa', 'Damage', 'Distention', 'Strain',
                         'MPa'])

    fld_factor = np.array([1., 1., 1., 1., 1e9, 1e6, 1e9, 1., 1., 1., 1e6])
    
    
    # In case for all the parameters 
    if param == 'all':
        
        # get all the field available for the model
        iSALEparam0 = model.fieldlist
        iSALEparam = []
    
        for param in iSALEparam0:
            if ((param[0].startswith('V_')) | (param[0].startswith('Tr'))):
                None
            else:
                iSALEparam.append(param[0])
                
        # get the lists
        param1 = []
        name = []
        cmap = []
        unit = []
        factor = []
        npathl = []
        
        for parami in iSALEparam:
            
            idx = np.where(fld == parami)[0][0]
        
            param1.append(fld_param[idx])
            name.append(fld_name[idx])
            cmap.append(fld_cmap[idx])
            unit.append(fld_unit[idx])
            factor.append(fld_factor[idx])
            
            
            for z in zoom_id:
                npath = paths + 'evolution/plots/' + fld_folder[idx] + '/' + z + '/'
                npathl.append(npath)
                
                if not os.path.exists(npath):
                    os.makedirs(npath)
                    
    
    else:
        idx = np.where(fld == param)[0][0]
    
        param1 = fld_param[idx]
        name = fld_name[idx]
        cmap = fld_cmap[idx]
        unit = fld_unit[idx]
        factor = fld_factor[idx]
        npathl = []
        
        for z in zoom_id:
            npath = paths + 'evolution/plots/' + fld_folder[idx] + '/' + z + '/'
            npathl.append(npath)
            
            if not os.path.exists(npath):
                os.makedirs(npath)

    return param1, name, cmap, unit, factor, npathl


'''
***********************************************************************
'''

def truncate(n, decimals=0):
    
    '''
    see website: https://realpython.com/python-rounding/
    
    >>> truncate(-5.963, 1)
    -5.9
    
    
    >>> truncate(-1374.25, -3)
    -1000.0
    '''
    
    multiplier = 10 ** float(decimals)
    return int(n * multiplier) / multiplier

'''
***********************************************************************
'''

def vmin_vmax(path1, mod1, fld_param, fld_factor, param):
    
    '''
    Problem with the total plastic strain and the acoustic fluidization strength
    
    '''
    
    
    # change to directory
    os.chdir(path1)
    
    # empty list
    vminlst = []
    vmaxlst = []
        
    # read step only one time
    step = mod1.readStep(fld_param, 1)
    
    
    for ifld, fld in enumerate(fld_param):
        
        if fld.startswith('Cm'):       
            vminlst.append(0.0)
            vmaxlst.append(1.0)
            
        elif fld.startswith('Al'):
            vminlst.append(1.00)
            vmaxlst.append(1.25)
            
        elif fld.startswith('Da'):
            vminlst.append(0.0)
            vmaxlst.append(1.0)
            
        elif fld.startswith('TP'):
            vminlst.append(1e-5)
            vmaxlst.append(1.0)
            
        elif fld.startswith('Pr'):
            vminlst.append(0.0)
            vmaxlst.append(1.0)
            
        elif fld.startswith('Yl'):
            vminlst.append(0.0)
            vmaxlst.append(1.0) 
        else:
            
            # could I automatically find vmiin and vmaax
            ixfld = np.where(step.cmc[0] >= 0.5) # at least contain 50% of material
            
            # automatically find vmin and vmax
            vminlst.append(0.0)
            vmaax = np.percentile(step.data[ifld][ixfld]/fld_factor[ifld], 70)
            
            
            # rounding 
            if vmaax >= 1000.:
                vmaaxg = truncate(vmaax, -2)
            elif ((vmaax >= 10.) & (vmaax < 1000.)):
                vmaaxg = truncate(vmaax, -1)
            elif ((vmaax >= 0.1) & (vmaax < 10.)):
                vmaaxg = truncate(vmaax, 1)
            else:
                vmaaxg = truncate(vmaax, 3)
            
            # if somehow the rounding does not work then just use 1
            if vmaaxg == 0.0:
                vmaxlst.append(1.0)
            else:
                vmaxlst.append(vmaaxg)
            
    return (vminlst, vmaxlst)
    
'''
***********************************************************************
'''


def field(path1, paths, norm, normpath, zoom_id, lbl, vmiin, vmaax, manualx, manualy, param = 'all'):
    '''
    param = 'Den'  
    path1 = '/uio/kant/geo-ceed-u1/nilscp/Desktop/stallo_work/benchmarkFI/CPPR/UAVG/CPPR10/CDILWPO/CDILWPO_L100_dev_larger_cells/'
    paths = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/CDILWPO_L100_dev_larger_cells/'
    normpath = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/CDILWPO_L100/'
    norm = 0
    zoom_id = ['close', 'mid', 'hires', 'all', 'manual'] # 'mid'; 'hires'; 'all' #depending on the zoom
    param = 'all'
    vmiin = [] # or vmiin = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1e-5, 0.0] # need to take into account the field factor
    vmaax = [] # or vmaax = [1.0, 1.0, 2000.0, 400.0, 1.0, 1.0, 1.0, 1.0, 1.25,1.0, 1.0]

    lbl = r"$\mathit{L} = 100 m"

    field(path1, paths, norm, normpath, zoom_id, lbl, vmiin, vmaax, param)
    
    I should replace vmiin and vmaax by percentile values that are rounded to the hundreds


    Field list:
    [['Cm1', 'Concentration, mat. 1'],
     ['Cm2', 'Concentration, mat. 2'],
     ['Den', 'Density'],
     ['Tmp', 'Temperature'],
     ['Pre', 'Pressure'],
     ['Sie', 'Specific internal energy'],
     ['Yld', 'Yield strength'],
     ['Dam', 'Damage'],
     ['Alp', 'Distension'],
     ['TPS', 'Total plastic strain (shear)'],
     ['YAc', 'Acoustic fluidisation strength'],
     ['V_x', 'Velocity (x)'],
     ['V_y', 'Velocity (y)'],
     ['Trx', 'Tracer x position'],
     ['Try', 'Tracer y position'],
     ['TrP', 'Tracer peak pressure'],
     ['TrT', 'Tracer peak temperature']]
    '''

    plt.ioff()  # figures does not pop up

    os.chdir(path1)
    mod1 = psp.opendatfile('jdata.dat')

    # modelname
    modelname = path1.split('/')[-2]

    # loading ot the data
    ###########################################################################
    if norm == 1:
        os.chdir(paths + 'transient/')

        # load data for the transient crater 1
        t_trg, da_tr1, Da_tr1, V_tr1, h_tr1, Dr_tr1, Vr_tr1, __ = np.loadtxt(
            modelname + '_tr.txt', delimiter='\t', comments='#')

        os.chdir(paths + 'final/')
        t_f, __, __, __, __, drim_f, __, __ = np.loadtxt(
            modelname + '_final.txt', delimiter='\t', comments='#') # put to 040 here
        
        rinner1 = drim_f/2.

    elif norm == 2:

        os.chdir(paths + 'transient/')

        # load data for the transient crater
        t_trg, __, __, __, __, __, __, __ = np.loadtxt(
            modelname + '_tr.txt', delimiter='\t', comments='#')

        modelnametr = normpath.split('/')[-2]
        os.chdir(normpath + 'transient/')

        # load data for the transient crater modtr
        t_tr1, da_tr1, Da_tr1, V_tr1, h_tr1, Dr_tr1, Vr_tr1, __ = np.loadtxt(
            modelnametr + '_tr.txt', delimiter='\t', comments='#')
        
        # is it normal that it only load the data for the norm path?
        # maybe I guess so
        os.chdir(normpath + 'final/')
        t_fnorm, __, __, __, __, drim_fnorm, __, __ = np.loadtxt(
            modelnametr + '_final.txt', delimiter='\t', comments='#') # put to 040 here
        
        rinner1 = drim_fnorm/2.

    else:
        os.chdir(paths + 'transient/')

        # load data for the transient crater 1
        t_trg, da_tr1, Da_tr1, V_tr1, h_tr1, Dr_tr1, Vr_tr1, __ = np.loadtxt(
            modelname + '_tr.txt', delimiter='\t', comments='#')

        os.chdir(paths + 'final/')
        t_f, __, __, __, __, drim_f, __, __ = np.loadtxt(
            modelname + '_final.txt', delimiter='\t', comments='#') # put to 040 here
            
        rinner1 = drim_f/2.

    ###########################################################################
    
    try:
        # choosing the degree of the zooming on the figure
        # get the extent
        extentx, extenty, zoom_id_up = zoom(zoom_id, norm, mod1, rinner1, manualx, manualy)
        
        # field list (should I make a function out of it?)
        fld_param, fld_name, fld_cmap, fld_unit, fld_factor, npath = field_definition(mod1, zoom_id_up, paths, param)

        # update label
        nlbl = []
        
        # small change (for case with only one selected parameter)
        if param != 'all':
            nlbl.append(lbl + ", " + fld_name + "$")
            fld_param = [fld_param]
            fld_factor = [fld_factor]
            fld_name = [fld_name]
            fld_cmap = [fld_cmap]
            fld_unit = [fld_unit]
        else:        
            for fname in fld_name:
                nlbl.append(lbl + ", " + fname + "$")
        
        # print fld_param
        print (fld_param)


        

        
        #vmin and vmax (update here)
        if len(vmiin) == 0:
            vmiin, vmaax = vmin_vmax(path1, mod1, fld_param, fld_factor, param)
        else:
            None
    
        # looping of
        os.chdir(path1)
        for i in range(mod1.nsteps):
            step = mod1.readStep(fld_param, i)
    
            t1 = step.time
            # plotting of the data
    
            # changing the vmin and vmax every step will change the value everytime
            # maybe I should define manually
            
            if norm >= 1:
                
                '''
                After the change including all parameters this statement will never
                be fullfill expect when param = 'Por'
                '''
                
                # don't like the 'Por' thing
                #if param == 'Por':
                #    cax = ax1.pcolormesh(mod1.x/rinner1, mod1.y/rinner1, (1. - (1./step.data[0]))*100.,
                #                         vmin=vmiin,
                #                         vmax=vmaax,
                #                         cmap=fld_cmap, zorder=2)
                
                count = 0
                
                for ifld, fld in enumerate(fld_param):
                    
                    for iz, z in enumerate(zoom_id_up):
                        
                        fig = plt.figure(figsize=(6, 6))
                        ax1 = fig.add_subplot(111)
                        
                        # plot data
                        cax = ax1.pcolormesh(mod1.x/rinner1, mod1.y/rinner1, step.data[ifld]/fld_factor[ifld],
                                                 vmin=vmiin[ifld],
                                                 vmax=vmaax[ifld],
                                                 cmap=fld_cmap[ifld], zorder=2)
        
                        ax1.contour(mod1.xc/rinner1, mod1.yc/rinner1,
                                step.cmc[0], 1, colors='k', linewidths=2, zorder=4)
                    
                    
        
                        for ax in [ax1]:
                            ax.minorticks_off()
                            ax.tick_params('both', labelsize=16, length=10,
                                           width=2., which='major')
                            
                        cbar = fig.colorbar(cax)
                        cbar.ax.tick_params(labelsize=15)
                        cbar.set_label('$' + fld_unit[ifld] + '$', labelpad=-
                                       42.5, y=1.07, fontsize=15, rotation=0)
                        
                        # set labels
                        ax1.set_xlabel(r"$x / R_{r}$", fontsize=20)
                        ax1.set_ylabel(r"$y / R_{r}$", fontsize=20)
                        
                        # set title            
                        ax1.set_title(r"$\mathit{t} = " + "{:.2f}".format(t1) + " " + " s, $" + " " +
                                      "$\zeta$ = " + "{:.2f}".format(t1/t_trg), position=(0.5, 0.9), fontsize=18)
                        
                        # related to the zoom
                        ax1.set_xlim(extentx[iz])
                        for ax in [ax1]:
                            ax.set_ylim(extenty[iz])
                        fig.tight_layout()
                        
                        st = fig.suptitle(nlbl[ifld], fontsize=20)
                        st.set_y(0.97)
                        st.set_x(0.5)
                        fig.subplots_adjust(top=0.9)
                            
                                                    
                        if norm == 1:
                            figtitle = 'itself_norm_' + modelname + \
                                '_' + str(int(i)).zfill(3) + ".png"
                            fig.savefig(npath[count] + figtitle, dpi=300)
                            
                        elif norm == 2:
                            figtitle = 'norm_' + modelname + \
                                '_' + str(int(i)).zfill(3) + ".png"
                            fig.savefig(npath[count] + figtitle, dpi=300)
                            
                        plt.close()
                        
                        count = count + 1
    
    
            else:
                count = 0
                
                for ifld, fld in enumerate(fld_param):
                    
                    for iz, z in enumerate(zoom_id_up):
                        
                        fig = plt.figure(figsize=(6, 6))
                        ax1 = fig.add_subplot(111)
                        
                        # plot data
                        cax = ax1.pcolormesh(mod1.x, mod1.y, step.data[ifld]/fld_factor[ifld],
                                                 vmin=vmiin[ifld],
                                                 vmax=vmaax[ifld],
                                                 cmap=fld_cmap[ifld], zorder=2)
        
                        ax1.contour(mod1.xc, mod1.yc,
                                step.cmc[0], 1, colors='k', linewidths=2, zorder=4)
                    
                    
        
                        for ax in [ax1]:
                            ax.minorticks_off()
                            ax.tick_params('both', labelsize=16, length=10,
                                           width=2., which='major')
                            
                        cbar = fig.colorbar(cax)
                        cbar.ax.tick_params(labelsize=15)
                        cbar.set_label('$' + fld_unit[ifld] + '$', labelpad=-
                                       42.5, y=1.07, fontsize=15, rotation=0)
                        
                        # set labels
                        ax1.set_xlabel(r"$x$", fontsize=20)
                        ax1.set_ylabel(r"$y$", fontsize=20)
                        
                        # set title            
                        ax1.set_title(r"$\mathit{t} = " + "{:.2f}".format(t1) + " " + " s$", position=(0.5, 0.9), fontsize=18)
                        
                        # related to the zoom
                        ax1.set_xlim(extentx[iz])
                        for ax in [ax1]:
                            ax.set_ylim(extenty[iz])
                        fig.tight_layout()
                        
                        st = fig.suptitle(nlbl[ifld], fontsize=20)
                        st.set_y(0.97)
                        st.set_x(0.5)
                        fig.subplots_adjust(top=0.9)
                            
                        figtitle = modelname + '_' + str(int(i)).zfill(3) + ".png"
                        fig.savefig(npath[count] + figtitle, dpi=300)
                        
                            
                        plt.close()
                        
                        count = count + 1
    except:
        None

    mod1.closeFile()


'''
***********************************************************************
'''


def mainevo(folders, path1, paths, norm, normpath, zoom_id, param):
    
    '''
    path1 = '/uio/kant/geo-ceed-u1/nilscp/Desktop/stallo_work/benchmarkFI/CPPR/UAVG/CPPR10/'
    paths = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/'
    normpath = ''
    norm = 0
    zoom_id = ['close', 'all', 'mid', 'hires'] # 'mid'; 'hires'; 'all' #depending on the zoom
    param = 'all'
    lbl = r"$\mathit{L} = 100 m"
    L = [100]
    
    # maybe detect automatically from what stands behind L 
    
    Need to be tested
    
    
    '''
    os.chdir(path1)
    
    folders = glob.glob('*')
    
    for f in folders:
        os.chdir(path1 + '/' + f)
        modelnames = glob.glob('*')
        print (modelnames)
     
        for im, modelname in enumerate(modelnames):
            
            pathsa = paths + modelname + '/'
            path1a = path1 + f + '/' + modelname + '/'
            
            L = modelname.split('_L')[1]
            Ln = L.split('_')[0]
            
            #normpath should be the same
            lbla = r"$\mathit{L} = " + Ln + " m"
                        
            field(path1a, pathsa, norm, normpath, zoom_id, lbla, param = 'all')
        
        

def main(pathdata, folders, pathplots):
    '''
    description:
    routine that plots both the transient, excavated and final cavities.
    This routine is useful to see whether the different crater dimensions by
    the scripts in crater.py and ejecta.py managed to perform well (i.e., this is
    visual quality assesment of the results)

    inputs:
    pathdata: path where the text files have been stored (text files that are obtained
    from running the main.py script)

    folders: all modelcases that need to be plotted

    pathplots: saving plot directory
    '''

    for modelname in folders:
        transient(pathdata, modelname, pathplots)
        final(pathdata, modelname, pathplots)
        excavated(pathdata, modelname, pathplots)

    print ("ROCK'N ROLL")


'''
***********************************************************************
'''


def transient(pathdata, modelname, pathplots):
    '''
    description:
    routine that plots transient crater dimensions

    inputs:
    pathdata: path where the text files have been stored (text files that are obtained
    from running the main.py script)

    folders: all modelcases that need to be plotted

    pathplots: saving plot directory

    example:
    pathdata = '/media/nilscp/Zell/Collapse/data/'
    modelname = 'C00P20F08_L250'
    pathplots = '/media/nilscp/Zell/Collapse/rplots/'
    transient(pathdata,modelname,pathplots)
    '''

    # should be outside of the loop
    if not os.path.exists(pathplots):
        os.makedirs(pathplots)

    os.chdir(pathdata + modelname + '/transient/')

    t, da, Da, V, h, Dr, Vr, __ = np.loadtxt(
        modelname + '_tr.txt', delimiter='\t', comments='#')
    dataXY = np.loadtxt(modelname + '_XYtransientprofile.txt',
                        delimiter='\t', comments='#')
    X = dataXY[:, 0]
    Y = dataXY[:, 1]

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111)
    ax.plot(X, Y, "ro", zorder=2)
    ax.hlines(0, np.min(X), np.max(X), 'k', linewidth=3)
    ax.plot(Da/2., 0, 'yo', ms=10, label='Ra: apparent radius')
    ax.plot(Dr/2., h, 'co', ms=10, label='Drim and hrim')
    ax.hlines(-da, np.min(X), np.max(X), 'b',
              linewidth=3, label='da: apparent depth')
    ax.legend(loc='center right')
    ax.set_xlabel('X (m)', fontsize=18)
    ax.set_ylabel('Y (m)', fontsize=18)
    ax.tick_params('both', labelsize=16, length=10, width=1., which='major')
    fig.tight_layout()
    st = fig.suptitle(modelname + ' transient profile', fontsize=18)
    st.set_y(0.95)
    fig.subplots_adjust(top=0.9)
    figtitle = modelname + ' transient_profile.png'
    fig.savefig(pathplots+figtitle, dpi=300)
    plt.close()


'''
***********************************************************************
'''


def final(pathdata, modelname, pathplots):
    '''
    description:
    routine that plots final crater dimensions

    inputs:
    pathdata: path where the text files have been stored (text files that are obtained
    from running the main.py script)

    folders: all modelcases that need to be plotted

    pathplots: saving plot directory

    example:
    pathdata = '/media/nilscp/Zell/Collapse/data/'
    modelname = 'C00P20F08_L250'
    pathplots = '/media/nilscp/Zell/Collapse/rplots/'
    final(pathdata,modelname,pathplots)
    '''

    # should be outside of the loop
    if not os.path.exists(pathplots):
        os.makedirs(pathplots)

    os.chdir(pathdata + modelname + '/final/')

    t, da, Da, V, h, Dr, Vr = np.loadtxt(
        modelname + '_final.txt', delimiter='\t', comments='#')
    dataXY = np.loadtxt(modelname + '_XYfinalprofile.txt',
                        delimiter='\t', comments='#')
    X = dataXY[:, 0]
    Y = dataXY[:, 1]

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111)
    ax.plot(X, Y, "ro", zorder=2)
    ax.hlines(0, np.min(X), np.max(X), 'k', linewidth=3)
    ax.plot(Da/2., 0, 'yo', ms=10, label='Ra: apparent radius')
    ax.plot(Dr/2., h, 'co', ms=10, label='Drim and hrim')
    ax.hlines(-da, np.min(X), np.max(X), 'b',
              linewidth=3, label='da: apparent depth')
    ax.legend(loc='center right')
    ax.set_xlabel('X (m)', fontsize=18)
    ax.set_ylabel('Y (m)', fontsize=18)
    ax.tick_params('both', labelsize=16, length=10, width=1., which='major')
    fig.tight_layout()
    st = fig.suptitle(modelname + ' final profile', fontsize=18)
    st.set_y(0.95)
    fig.subplots_adjust(top=0.9)
    figtitle = modelname + ' final_profile.png'
    fig.savefig(pathplots+figtitle, dpi=300)
    plt.close()


'''
***********************************************************************
'''


def excavated(pathdata, modelname, pathplots):
    '''
    description:
    routine that plots the excavated crater dimensions

    inputs:
    pathdata: path where the text files have been stored (text files that are obtained
    from running the main.py script)

    folders: all modelcases that need to be plotted

    pathplots: saving plot directory

    example:
    pathdata = '/media/nilscp/Zell/Collapse/data/'
    modelname = 'C00P20F08_L250'
    pathplots = '/media/nilscp/Zell/Collapse/rplots/'
    excavated(pathdata,modelname,pathplots)
    '''

    # should be outside of the loop
    if not os.path.exists(pathplots):
        os.makedirs(pathplots)

    os.chdir(pathdata + modelname + '/excavated/')

    # load excavated data
    # possible miss something at the end
    Ve, de, De = np.loadtxt(modelname + '_excavated.txt',
                            delimiter='\t', comments="#")
    Z = np.loadtxt(modelname + '_tracersXY_excavated.txt', delimiter='\t',
                   comments="#")  # possible miss something at the end
    X = Z[:, 0]
    Y = Z[:, 1]

    # load transient profile
    os.chdir(pathdata + modelname + '/transient/')
    dataXY = np.loadtxt(modelname + '_XYtransientprofile.txt',
                        delimiter='\t', comments='#')
    X2 = dataXY[:, 0]
    Y2 = dataXY[:, 1]

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111)
    ax.plot(X, Y, "bo")
    ax.plot(X2, Y2, "ro")
    ax.hlines(de, np.min(X2), np.max(X2), 'b', linewidth=3,
              label='de: apparent excavated depth')
    ax.plot(De/2., 0, 'yo', ms=10, label='Re: apparent excavated radius')
    ax.legend(loc='lower right')
    ax.set_xlabel('X (m)', fontsize=18)
    ax.set_ylabel('Y (m)', fontsize=18)
    ax.tick_params('both', labelsize=16, length=10, width=1., which='major')
    fig.tight_layout()
    st = fig.suptitle(
        modelname + ' excavated and transient profile', fontsize=18)
    st.set_y(0.95)
    fig.subplots_adjust(top=0.9)
    figtitle = modelname + ' excavated_profile.png'
    fig.savefig(pathplots+figtitle, dpi=300)
    plt.close()
    
'''
***********************************************************************
'''

'\n***********************************************************************\n'

### Plotting the evolution of craters at different zooms but also for different parameters such as the Pressure, the density and ++
Example for three models within one specific folder

In [15]:
path = '/uio/kant/geo-ceed-u1/nilscp/Desktop/stallo_work/benchmarkFI/CPPR/UAVG/CPPR10/CDILWPO/'
pathdata = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/'
pathplots = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/plots/'

os.chdir(path)
models = glob.glob('*NOP*')

print (models)

['CDILWPO_L100NOPOR']


Here are some python parameters that need to be defined

In [16]:
mode = 1
poros = 1  # if porous = 1, non porous = 0
id_mat = 0  # 0 = everything, 2 only lower material, especially practical in layering modelling
transient = 0  # 0 transient at max volume, 1: transient at maximum depth
thresholdf = 0.01 # for crater ejecta
g = 1.62 # gravity of planetary body

Plotting of crater dimensions during crater evolutionm at the transient and final crater diameter

In [ ]:
cr.main(path, models, pathdata, mode, id_mat, transient)

normpath = ''
norm = 0
zoom_id = ['close', 'mid', 'hires', 'all'] # 'mid'; 'hires'; 'all'; 'manual' #depending on the zoom
param = 'all'
vmiin = [] # or vmiin = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1e-5, 0.0] # need to take into account the field factor
vmaax = [] # or vmaax = [1.0, 1.0, 2000.0, 400.0, 1.0, 1.0, 1.0, 1.0, 1.25,1.0, 1.0]

lbl = r"$\mathit{L} = 100 m"

In [18]:
manualx = 0.0 # not taken into account here  
manualy = 0.0 # not taken into account here

for m in models:
    path1 = '/uio/kant/geo-ceed-u1/nilscp/Desktop/stallo_work/benchmarkFI/CPPR/UAVG/CPPR10/CDILWPO/' + m + '/'
    paths = '/run/media/nilscp/Squall/benchmarkFI/CPPR10/data/' + m + '/'

    field(path1, paths, norm, normpath, zoom_id, lbl, vmiin, vmaax, manualx, manualy, param)

Opened iSALE data file 'jdata.dat', with 203 time steps
['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc']
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 1 (2.501e-01 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 0 (0.000e+00 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 1 (2.501e-01 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 2 (5.000e-01 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 3 (7.501e-01 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 4 (1.000e+00 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Dam', 'Alp', 'TPS', 'YAc'] for timestep 5 (1.250e+00 s)
Read in ['Cm1', 'Cm2', 'Den', 'Tmp', 'Pre', 'Sie', 'Yld', 'Da

Plotting of all the other parameters